In [31]:
import torch
import math
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin
from sklearn.utils.multiclass import check_classification_targets

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.utils.data import DataLoader
from tqdm import tqdm
from imodelsx.kan.kan_modules import KANModule, KANGAMModule


from typing import List

#import  psutil
#import  os
#import  time
import  joblib
import  warnings
import  pandas as pd
import  seaborn as sns
import  snap7
import  ctypes
import  struct

import  matplotlib.pyplot as plt

from    torch.utils.data import DataLoader, TensorDataset

from    sklearn.metrics import confusion_matrix, classification_report

from    sklearn.preprocessing import LabelEncoder
from    sklearn.model_selection import GridSearchCV
from    sklearn.preprocessing import StandardScaler
from    sklearn.preprocessing import MinMaxScaler
from    sklearn.model_selection import ParameterGrid
from    snap7.type import S7DataItem, Area, WordLen
from    snap7.util import *
from    imblearn.over_sampling import SMOTE
from    imblearn.under_sampling import RandomUnderSampler
from    imblearn.pipeline import Pipeline
from    imblearn.combine import SMOTETomek 


warnings.filterwarnings("ignore")

if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

print(device)

cuda


# Implementação Efficient KAN

In [32]:

class KANLinearModule(torch.nn.Module):
    def __init__(
        self,
        in_features,
        out_features,
        grid_size=5,
        spline_order=3,
        scale_noise=0.1,
        scale_base=1.0,
        scale_spline=1.0,
        enable_standalone_scale_spline=True,
        base_activation=torch.nn.SiLU,
        grid_eps=0.02,
        grid_range=[-1, 1],
    ):
        super(KANLinearModule, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.grid_size = grid_size
        self.spline_order = spline_order

        h = (grid_range[1] - grid_range[0]) / grid_size
        grid = (
            (
                torch.arange(-spline_order, grid_size + spline_order + 1) * h
                + grid_range[0]
            )
            .expand(in_features, -1)
            .contiguous()
        )
        self.register_buffer("grid", grid)

        self.base_weight = torch.nn.Parameter(
            torch.Tensor(out_features, in_features))
        self.spline_weight = torch.nn.Parameter(
            torch.Tensor(out_features, in_features, grid_size + spline_order)
        )
        if enable_standalone_scale_spline:
            self.spline_scaler = torch.nn.Parameter(
                torch.Tensor(out_features, in_features)
            )

        self.scale_noise = scale_noise
        self.scale_base = scale_base
        self.scale_spline = scale_spline
        self.enable_standalone_scale_spline = enable_standalone_scale_spline
        self.base_activation = base_activation()
        self.grid_eps = grid_eps

        self.reset_parameters()

    def reset_parameters(self):
        torch.nn.init.kaiming_uniform_(
            self.base_weight, a=math.sqrt(5) * self.scale_base)
        with torch.no_grad():
            noise = (
                (
                    torch.rand(self.grid_size + 1,
                               self.in_features, self.out_features)
                    - 1 / 2
                )
                * self.scale_noise
                / self.grid_size
            )
            self.spline_weight.data.copy_(
                (self.scale_spline if not self.enable_standalone_scale_spline else 1.0)
                * self.curve2coeff(
                    self.grid.T[self.spline_order: -self.spline_order],
                    noise,
                )
            )
            if self.enable_standalone_scale_spline:
                # torch.nn.init.constant_(self.spline_scaler, self.scale_spline)
                torch.nn.init.kaiming_uniform_(
                    self.spline_scaler, a=math.sqrt(5) * self.scale_spline)

    def b_splines(self, x: torch.Tensor):
        """
        Compute the B-spline bases for the given input tensor.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_features).

        Returns:
            torch.Tensor: B-spline bases tensor of shape (batch_size, in_features, grid_size + spline_order).
        """
        assert x.dim() == 2 and x.size(1) == self.in_features

        grid: torch.Tensor = (
            self.grid
        )  # (in_features, grid_size + 2 * spline_order + 1)
        x = x.unsqueeze(-1)
        bases = ((x >= grid[:, :-1]) & (x < grid[:, 1:])).to(x.dtype)
        for k in range(1, self.spline_order + 1):
            bases = (
                (x - grid[:, : -(k + 1)])
                / (grid[:, k:-1] - grid[:, : -(k + 1)])
                * bases[:, :, :-1]
            ) + (
                (grid[:, k + 1:] - x)
                / (grid[:, k + 1:] - grid[:, 1:(-k)])
                * bases[:, :, 1:]
            )

        assert bases.size() == (
            x.size(0),
            self.in_features,
            self.grid_size + self.spline_order,
        )
        return bases.contiguous()

    def curve2coeff(self, x: torch.Tensor, y: torch.Tensor):
        """
        Compute the coefficients of the curve that interpolates the given points.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, in_features).
            y (torch.Tensor): Output tensor of shape (batch_size, in_features, out_features).

        Returns:
            torch.Tensor: Coefficients tensor of shape (out_features, in_features, grid_size + spline_order).
        """
        assert x.dim() == 2 and x.size(1) == self.in_features
        assert y.size() == (x.size(0), self.in_features, self.out_features)

        A = self.b_splines(x).transpose(
            0, 1
        )  # (in_features, batch_size, grid_size + spline_order)
        B = y.transpose(0, 1)  # (in_features, batch_size, out_features)
        solution = torch.linalg.lstsq(
            A, B
        ).solution  # (in_features, grid_size + spline_order, out_features)
        result = solution.permute(
            2, 0, 1
        )  # (out_features, in_features, grid_size + spline_order)

        assert result.size() == (
            self.out_features,
            self.in_features,
            self.grid_size + self.spline_order,
        )
        return result.contiguous()

    @property
    def scaled_spline_weight(self):
        return self.spline_weight * (
            self.spline_scaler.unsqueeze(-1)
            if self.enable_standalone_scale_spline
            else 1.0
        )

    def forward(self, x: torch.Tensor):
        assert x.dim() == 2 and x.size(1) == self.in_features

        base_output = F.linear(self.base_activation(x), self.base_weight)
        spline_output = F.linear(
            self.b_splines(x).view(x.size(0), -1),
            self.scaled_spline_weight.view(self.out_features, -1),
        )
        return base_output + spline_output

    @torch.no_grad()
    def update_grid(self, x: torch.Tensor, margin=0.01):
        assert x.dim() == 2 and x.size(1) == self.in_features
        batch = x.size(0)

        splines = self.b_splines(x)  # (batch, in, coeff)
        splines = splines.permute(1, 0, 2)  # (in, batch, coeff)
        orig_coeff = self.scaled_spline_weight  # (out, in, coeff)
        orig_coeff = orig_coeff.permute(1, 2, 0)  # (in, coeff, out)
        unreduced_spline_output = torch.bmm(
            splines, orig_coeff)  # (in, batch, out)
        unreduced_spline_output = unreduced_spline_output.permute(
            1, 0, 2
        )  # (batch, in, out)

        # sort each channel individually to collect data distribution
        x_sorted = torch.sort(x, dim=0)[0]
        grid_adaptive = x_sorted[
            torch.linspace(
                0, batch - 1, self.grid_size + 1, dtype=torch.int64, device=x.device
            )
        ]

        uniform_step = (x_sorted[-1] - x_sorted[0] +
                        2 * margin) / self.grid_size
        grid_uniform = (
            torch.arange(
                self.grid_size + 1, dtype=torch.float32, device=x.device
            ).unsqueeze(1)
            * uniform_step
            + x_sorted[0]
            - margin
        )

        grid = self.grid_eps * grid_uniform + \
            (1 - self.grid_eps) * grid_adaptive
        grid = torch.concatenate(
            [
                grid[:1]
                - uniform_step
                * torch.arange(self.spline_order, 0, -1,
                               device=x.device).unsqueeze(1),
                grid,
                grid[-1:]
                + uniform_step
                * torch.arange(1, self.spline_order + 1,
                               device=x.device).unsqueeze(1),
            ],
            dim=0,
        )

        self.grid.copy_(grid.T)
        self.spline_weight.data.copy_(
            self.curve2coeff(x, unreduced_spline_output))

    def regularization_loss(self, regularize_activation=1.0, regularize_entropy=1.0):
        """
        Compute the regularization loss.

        This is a dumb simulation of the original L1 regularization as stated in the
        paper, since the original one requires computing absolutes and entropy from the
        expanded (batch, in_features, out_features) intermediate tensor, which is hidden
        behind the F.linear function if we want an memory efficient implementation.

        The L1 regularization is now computed as mean absolute value of the spline
        weights. The authors implementation also includes this term in addition to the
        sample-based regularization.
        """
        l1_fake = self.spline_weight.abs().mean(-1)
        regularization_loss_activation = l1_fake.sum()
        p = l1_fake / regularization_loss_activation
        regularization_loss_entropy = -torch.sum(p * p.log())
        return (
            regularize_activation * regularization_loss_activation
            + regularize_entropy * regularization_loss_entropy
        )


class KANModule(torch.nn.Module):
    def __init__(
        self,
        layers_hidden,
        grid_size=5,
        spline_order=3,
        scale_noise=0.1,
        scale_base=1.0,
        scale_spline=1.0,
        base_activation=torch.nn.SiLU,
        grid_eps=0.02,
        grid_range=[-1, 1],
    ):
        super(KANModule, self).__init__()
        self.grid_size = grid_size
        self.spline_order = spline_order

        self.layers = torch.nn.ModuleList()
        for in_features, out_features in zip(layers_hidden, layers_hidden[1:]):
            self.layers.append(
                KANLinearModule(
                    in_features,
                    out_features,
                    grid_size=grid_size,
                    spline_order=spline_order,
                    scale_noise=scale_noise,
                    scale_base=scale_base,
                    scale_spline=scale_spline,
                    base_activation=base_activation,
                    grid_eps=grid_eps,
                    grid_range=grid_range,
                )
            )

    def forward(self, x: torch.Tensor, update_grid=False):
        for layer in self.layers:
            if update_grid:
                layer.update_grid(x)
            x = layer(x)
        return x

    def regularization_loss(self, regularize_activation=1.0, regularize_entropy=1.0):
        return sum(
            layer.regularization_loss(
                regularize_activation, regularize_entropy)
            for layer in self.layers
        )


class KANGAMModule(torch.nn.Module):
    '''Learn a KAN model on each individual input feature
    '''

    def __init__(self, num_features, layers_hidden: List[int], n_classes, **kwargs):
        super(KANGAMModule, self).__init__()
        self.models = torch.nn.ModuleList([
            KANModule(
                layers_hidden=[1] + layers_hidden + [1],
                **kwargs)
            for _ in range(num_features)
        ])
        self.linear = torch.nn.Linear(num_features, n_classes)

    def forward(self, x: torch.Tensor, update_grid=False):

        features = torch.stack(
            [model(x[:, i:i + 1], update_grid)
             for i, model in enumerate(self.models)],
            dim=1)

        features = features.view(x.size(0), -1)
        return self.linear(features)

    def regularization_loss(self, regularize_activation=1.0, regularize_entropy=1.0, regularize_ridge=1.0):
        return sum(
            layer.regularization_loss(
                regularize_activation, regularize_entropy)
            for model in self.models
            for layer in model.layers
        ) + regularize_ridge * self.linear.weight.norm(p=2)

# KAN Classifier

In [33]:
class KAN(BaseEstimator):
    def __init__(self,
                 hidden_layer_size: int = 64,
                 hidden_layer_sizes: List[int] = None,
                 regularize_activation: float = 1.0, regularize_entropy: float = 1.0, regularize_ridge: float = 0.0,
                 test_size=0.2, random_state=42, shuffle=True,
                 device: str = 'cpu',
                 **kwargs):
        '''
        Params
        ------
        hidden_layer_size : int
            If int, number of neurons in the hidden layer (assumes single hidden layer)
        hidden_layer_sizes: List with length (n_layers - 2)
            The ith element represents the number of neurons in the ith hidden layer.
            If this is passed, will override hidden_layer_size
            e.g. [32, 64] would have a layer with 32 hidden units followed by a layer with 64 hidden units
            (input and output shape are inferred by the data passed)
        regularize_activation: float
            Activation regularization parameter
        regularize_entropy: float
            Entropy regularization parameter
        regularize_ridge: float
            Ridge regularization parameter (only applies to KANGAM)
        kwargs can be any of these more detailed KAN parameters
            grid_size=5,
            spline_order=3,
            scale_noise=0.1,
            scale_base=1.0,
            scale_spline=1.0,
            base_activation=torch.nn.SiLU,
            grid_eps=0.02,
            grid_range=[-1, 1],
        '''
        if hidden_layer_sizes is not None:
            self.hidden_layer_sizes = hidden_layer_sizes
        else:
            self.hidden_layer_sizes = [hidden_layer_size]
        self.device = device
        self.regularize_activation = regularize_activation
        self.regularize_entropy = regularize_entropy
        self.regularize_ridge = regularize_ridge
        self.kwargs = kwargs

    def fit(self, X, y, batch_size=512, lr=1e-3, weight_decay=1e-4, gamma=0.8):
        if isinstance(self, ClassifierMixin):
            check_classification_targets(y)
            self.classes_, y = np.unique(y, return_inverse=True)
            num_outputs = len(self.classes_)
            y = torch.tensor(y, dtype=torch.long)
        else:
            num_outputs = 1
            y = torch.tensor(y, dtype=torch.float32)
        X = torch.tensor(X, dtype=torch.float32)
        num_features = X.shape[1]

        if isinstance(self, (KANGAMClassifier, KANGAMRegressor)):
            self.model = KANGAMModule(
                num_features=num_features,
                layers_hidden=self.hidden_layer_sizes,
                n_classes=num_outputs,
                **self.kwargs
            ).to(self.device)
        else:
            self.model = KANModule(
                layers_hidden=[num_features] +
                self.hidden_layer_sizes + [num_outputs],
            ).to(self.device)

        X_train, X_tune, y_train, y_tune = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

        dset_train = torch.utils.data.TensorDataset(X_train, y_train)
        dset_tune = torch.utils.data.TensorDataset(X_tune, y_tune)
        loader_train = DataLoader(dset_train, batch_size=batch_size, shuffle=True)
        loader_tune = DataLoader(dset_tune, batch_size=batch_size, shuffle=False)

        optimizer = optim.AdamW(self.model.parameters(), lr=lr, weight_decay=weight_decay)
        scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)

        # Define loss
        if isinstance(self, ClassifierMixin):
            criterion = nn.CrossEntropyLoss()
        else:
            criterion = nn.MSELoss()
        tune_losses = []
        for epoch in tqdm(range(100)):
            self.model.train()
            for x, labs in loader_train:
                x = x.view(-1, num_features).to(self.device)
                optimizer.zero_grad()
                output = self.model(x).squeeze()
                loss = criterion(output, labs.to(self.device).squeeze())
                if isinstance(self, (KANGAMClassifier, KANGAMRegressor)):
                    loss += self.model.regularization_loss(
                        self.regularize_activation, self.regularize_entropy, self.regularize_ridge)
                else:
                    loss += self.model.regularization_loss(
                        self.regularize_activation, self.regularize_entropy)

                loss.backward()
                optimizer.step()

            # Validation
            self.model.eval()
            tune_loss = 0
            with torch.no_grad():
                for x, labs in loader_tune:
                    x = x.view(-1, num_features).to(self.device)
                    output = self.model(x).squeeze()
                    tune_loss += criterion(output,
                                           labs.to(self.device).squeeze()).item()
            tune_loss /= len(loader_tune)
            tune_losses.append(tune_loss)
            scheduler.step()

            # apply early stopping
            if len(tune_losses) > 3 and tune_losses[-1] > tune_losses[-2]:
                print("\tEarly stopping")
                return self

        return self

    @torch.no_grad()
    def predict(self, X):
        X = torch.tensor(X, dtype=torch.float32).to(self.device)
        output = self.model(X)
        if isinstance(self, ClassifierMixin):
            return self.classes_[output.argmax(dim=1).cpu().numpy()]
        else:
            return output.cpu().numpy()


class KANClassifier(KAN, ClassifierMixin):
    @torch.no_grad()
    def predict_proba(self, X):
        X = torch.tensor(X, dtype=torch.float32).to(self.device)
        output = self.model(X)
        return torch.nn.functional.softmax(output, dim=1).cpu().numpy()


class KANRegressor(KAN, RegressorMixin):
    pass


class KANGAMClassifier(KANClassifier):
    pass


class KANGAMRegressor(KANRegressor):
    pass

# Data Preparation

In [34]:
# dados importados da plataforma KAGGLE - https://www.kaggle.com/datasets/shivamb/machine-predictive-maintenance-classification
df = pd.read_csv("predictive_maintenance.csv")
df.head()


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Target,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,No Failure


In [35]:
df=df.drop(['Product ID','UDI','Target'],axis=1)

In [36]:
label_encoder = LabelEncoder()

# Perform label encoding
df['Failure Type_encoded'] = label_encoder.fit_transform(df['Failure Type'])

df['Failure Type_encoded'].value_counts()

''''
0 - 112  - heat dissipation
1 - 9652 - no Failure
2 - 78   - overstrain
3 - 95   - power failure
4 - 18   - random failure
5 - 45   - tool wear
'''


"'\n0 - 112  - heat dissipation\n1 - 9652 - no Failure\n2 - 78   - overstrain\n3 - 95   - power failure\n4 - 18   - random failure\n5 - 45   - tool wear\n"

In [37]:
df = pd.get_dummies(df, columns=['Type'], prefix='Type', drop_first=True)
df.head()


,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Failure Type,Failure Type_encoded,Type_L,Type_M
0,298.1,308.6,1551,42.8,0,No Failure,1,False,True
1,298.2,308.7,1408,46.3,3,No Failure,1,True,False
2,298.1,308.5,1498,49.4,5,No Failure,1,True,False
3,298.2,308.6,1433,39.5,7,No Failure,1,True,False
4,298.2,308.7,1408,40.0,9,No Failure,1,True,False


In [38]:
# Definição do conjunto de dados de entrada e do conjunto de dados de saída

X_raw = df.drop(['Failure Type','Failure Type_encoded'], axis=1).values
y_raw= df['Failure Type_encoded'].values

In [39]:
print('Total de amostras antes balanceamento:',df['Failure Type_encoded'].count())
print('Total de amostras por falha:\n\n', df['Failure Type_encoded'].value_counts())

Total de amostras antes balanceamento: 10000
Total de amostras por falha:

 Failure Type_encoded
1    9652
0     112
3      95
2      78
5      45
4      18
Name: count, dtype: int64


In [40]:
#implementação do método SMOTE no conunto de dados
smt=SMOTETomek(sampling_strategy='auto',random_state=42)
X_resampled, y_resampled = smt.fit_resample(X_raw, y_raw)

# Convertendo para dataframe do pandas
df_resampled = pd.DataFrame(X_resampled, columns=[f'feature_{i}' for i in range(X_resampled.shape[1])])
df_resampled['Type_encoded'] = y_resampled 

df_resampled.head()

#distribuição das classes após balanceamento
print('Total de amostras após balanceamento:',df_resampled['Type_encoded'].count())
print('Total de amostras por falha:\n\n', df_resampled['Type_encoded'].value_counts())

Total de amostras após balanceamento: 57868
Total de amostras por falha:

 Type_encoded
2    9652
0    9649
5    9647
3    9645
4    9645
1    9630
Name: count, dtype: int64


In [41]:
df_resampled = df_resampled.rename(columns={'feature_0':'Temp_Ar'})
df_resampled = df_resampled.rename(columns={'feature_1':'Temp_Pr'})
df_resampled = df_resampled.rename(columns={'feature_2':'Vel_Spindle'})
df_resampled = df_resampled.rename(columns={'feature_3':'Torque'})
df_resampled = df_resampled.rename(columns={'feature_4':'Desg_Ferr'})
df_resampled = df_resampled.rename(columns={'feature_5':'Mat_L'})
df_resampled = df_resampled.rename(columns={'feature_6':'Mat_M'})

df_resampled = df_resampled.rename(columns={'Type_encoded':'Tipo_Falha'})

df_resampled.head()


,Temp_Ar,Temp_Pr,Vel_Spindle,Torque,Desg_Ferr,Mat_L,Mat_M,Tipo_Falha
0,298.1,308.6,1551.0,42.8,0.0,0.0,1.0,1
1,298.2,308.7,1408.0,46.3,3.0,1.0,0.0,1
2,298.1,308.5,1498.0,49.4,5.0,1.0,0.0,1
3,298.2,308.6,1433.0,39.5,7.0,1.0,0.0,1
4,298.2,308.7,1408.0,40.0,9.0,1.0,0.0,1


In [42]:
X_ = df_resampled.drop(['Tipo_Falha'], axis="columns")
scaler = MinMaxScaler(feature_range=(-1, 1))

X   = scaler.fit_transform(X_)    
joblib.dump(scaler, 'minmax_scaler.pkl')

y = df_resampled['Tipo_Falha']

# Grid Search

In [53]:
def grid_search(X_train, y_train, X_test, y_test, param_grid, device):

    for h_layer_size in param_grid['hidden_layer_size']:
        for r_activation in param_grid['regularize_activation']:
            for r_entropy in param_grid['regularize_entropy']:
                for r_ridge in param_grid['regularize_ridge']:
                    for b_size in param_grid['batch_size']:
                        for w_decay in param_grid['weight_decay']:
                            for g in param_grid['gamma']:
                                for l in param_grid['lr']:

                                    model = KANClassifier(hidden_layer_size     = h_layer_size,
                                                          device                = device,
                                                          regularize_activation = r_activation,
                                                          regularize_entropy    = r_entropy,
                                                          regularize_ridge      = r_ridge)                                   

                                    print(f'Executando: layers: {h_layer_size}, activation: {r_activation}, entropy: {r_entropy}, ridge: {r_ridge}, batch size: {b_size}, decay: {w_decay}, gamma: {g}, learning reate: {l}')
                                                                       
                                    model.fit(X_train,y_train,batch_size = b_size,lr = l,weight_decay = w_decay,gamma = g)

                                    y_train_pred = model.predict(X_train)
                                    f1_train = f1_score(y_train, y_train_pred, average='weighted')

                                    y_test_pred = model.predict(X_test)
                                    f1_test = f1_score(y_test, y_test_pred, average='weighted')

                                    acc = accuracy_score(y_test, y_test_pred)

                                    result = {'f1_train' : f1_train,'f1_test'  : f1_test, 'accuracy' : acc}

                                    results.append(result)   
    return results


In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [55]:
param_grid = {
    'hidden_layer_size'     : [256, 512, 1024],
    'regularize_activation' : [0.4, 0.5, 0.6],
    'regularize_entropy'    : [0.4, 0.5, 0.6], 
    'regularize_ridge'      : [0.05, 0.1, 0.2], 
    'batch_size'            : [64, 128, 512],    
    'weight_decay'          : [0.005, 0.01, 0.03],
    'gamma'                 : [0.4, 0.5, 0.6],
    'lr'                    : [0.05, 0.07, 0.1]
    }


grid_results = grid_search(X_train, y_train, X_test, y_test, param_grid, device)

best_result_Accuracy        = max(results, key=lambda x: x['accuracy'])
best_result_F1_Score_train  = max(results, key=lambda x: x['f1_train'])
best_result_F1_Score_test   = max(results, key=lambda x: x['f1_test'])


Executando: layers: 256, activation: 0.4, entropy: 0.4, ridge: 0.05, batch size: 64, decay: 0.005, gamma: 0.4, learning reate: 0.05


AttributeError: cannot assign module before Module.__init__() call

In [ ]:
'''''

    for par in ParameterGrid(param_grid):

        print('Configuração em execução: ',par)

        model = KANClassifier(hidden_layer_size     = param_grid['hidden_layer_size'],
                              device                = device,
                              regularize_activation = param_grid['regularize_activation'],
                              regularize_entropy    = param_grid['regularize_entropy'],
                              regularize_ridge      = param_grid['regularize_ridge'])


        model.fit(X_train, y_train, batch_size=param_grid['batch_size'], lr= param_grid['lr'], weight_decay=['weight_decay'], gamma=['gamma'])

        y_train_pred = model.predict(X_train)
        f1_train = f1_score(y_train, y_train_pred, average='weighted')

        y_test_pred = model.predict(X_test)
        f1_test = f1_score(y_test, y_test_pred, average='weighted')

        acc = accuracy_score(y_test, y_test_pred)

        result = { 
                'f1_train' : f1_train,
                'f1_test'  : f1_test,
                'accuracy' : acc
        }

        results.append(result)
    '''